# <font color=red>모델 구성(Forward)</font>

### 원본

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import dgl

from libs.layers import GraphConvolution
from libs.layers import GraphIsomorphism
from libs.layers import GraphIsomorphismEdge
from libs.layers import GraphAttention


class MyModel(nn.Module):
	def __init__(
			self, 
			model_type,
			num_layers=4, 
			hidden_dim=64,
			num_heads=4, # Only used for GAT
			dropout_prob=0.2,
			bias_mlp=True,
			readout='sum',
			act=F.relu,
			initial_node_dim=58,
			initial_edge_dim=6,
			is_classification=False,
		):
		super().__init__()

		self.num_layers = num_layers
		self.embedding_node = nn.Linear(initial_node_dim, hidden_dim, bias=False)
		self.embedding_edge = nn.Linear(initial_edge_dim, hidden_dim, bias=False)
		self.readout = readout

		self.mp_layers = torch.nn.ModuleList()
		for _ in range(self.num_layers):
			mp_layer = None
			if model_type == 'gcn':
				mp_layer = GraphConvolution(
					hidden_dim=hidden_dim,
					dropout_prob=dropout_prob,
					act=act,
				)
			elif model_type == 'gin':
				mp_layer = GraphIsomorphism(
					hidden_dim=hidden_dim,
					dropout_prob=dropout_prob,
					act=act,
					bias_mlp=bias_mlp
				)
			elif model_type == 'gin_e':
				mp_layer = GraphIsomorphismEdge(
					hidden_dim=hidden_dim,
					dropout_prob=dropout_prob,
					act=act,
					bias_mlp=bias_mlp
				)
			elif model_type == 'gat':
				mp_layer = GraphAttention(
					hidden_dim=hidden_dim,
					num_heads=num_heads,
					dropout_prob=dropout_prob,
					act=act,
					bias_mlp=bias_mlp
				)
			else:
				raise ValueError('Invalid model type: you should choose model type in [gcn, gin, gin_e, gat, ggnn]')
			self.mp_layers.append(mp_layer)

		self.linear_out = nn.Linear(hidden_dim, 1, bias=False)

		self.is_classification = is_classification
		if self.is_classification:
			self.sigmoid = F.sigmoid
	

	def forward(
			self, 
			graph,
			training=False,
		):
		h = self.embedding_node(graph.ndata['h'].float())
		e_ij = self.embedding_edge(graph.edata['e_ij'].float())
		graph.ndata['h'] = h
		graph.edata['e_ij'] = e_ij

		# Update the node features
		for i in range(self.num_layers):
			graph = self.mp_layers[i](
				graph=graph,
				training=training
			)

		# Aggregate the node features and apply the last linear layer to compute the logit
		out = dgl.readout_nodes(graph, 'h', op=self.readout)
		out = self.linear_out(out)

		if self.is_classification:
			out = self.sigmoid(out)
		return out


### 주석

In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import dgl

from layers import GraphConvolution
from layers import GraphIsomorphism
from layers import GraphIsomorphismEdge
from layers import GraphAttention

# 자식 클래스 MyModel 선언 (부모 클래스 torch.nn.Module를 상속 받음)
class MyModel(nn.Module):
    def __init__(
            self, 
            model_type,    # model type은 그때 그때 설정
            num_layers=4,  # node (embedding?) layer: 4
            hidden_dim=64, # hidden dimension: 64
            num_heads=4,   ### Only used for GAT
            dropout_prob=0.2,
            bias_mlp=True, # MLP에 들어가는 bias 사용
            readout='sum',
            act=F.relu,    # non-linear activation: ReLU
            initial_node_dim=58,  # 맨처음 node dimention: 58 (node 하나당 dimension)
            initial_edge_dim=6,   # 맨처음 edge dimention: 6
            is_classification=False,
        ):
		 # 부모클래스에 없는 다른 작업 추가
        super().__init__()
        
		### 정의 
        self.num_layers = num_layers
        # Linear layer(MLP)로 embedding: initial node feature의 dimension을 64로 맞춰주기 위해서 node feature를 embedding하는 layer 정의 (linear transformation)
        # dimension만 바꾸면 되기 때문에 bias parameter를 사용하여 overfitting에 취약하게 만들 수 있어서 사용하지 않음. 
        self.embedding_node = nn.Linear(initial_node_dim, hidden_dim, bias=False)
        self.embedding_edge = nn.Linear(initial_edge_dim, hidden_dim, bias=False)
        # sum/ max/ mean readout할 지
        self.readout = readout
        
		# self.mp_layers 정의 후        
        # message passing layer: 특정 node를 설명하기 위한 재료로 그 node의 neighborhood의 특징을 모으는 과정이 바로 Message Passing
        # ModuleList(): 이것은 간단히 말해 nn.Module을 리스트로 정리하는 방법이다.
        # 각 레이어를 리스트에 전달하고 레이어의 iterator를 만든다. 덕분에 forward처리를 간단하게 할 수 있다는 듯 하다.
        # 밑에서 append해줄 mp_layer 하나하나가 torch.nn.Module 이라는 뜻.
        self.mp_layers = torch.nn.ModuleList()
        
        # model_type(GCN or GIN or GIN_E or GAT) 중 하나를 num_layer=4번
        for _ in range(self.num_layers):
            mp_layer = None
            if model_type == 'gcn':
                mp_layer = GraphConvolution(
                    hidden_dim=hidden_dim,
                    dropout_prob=dropout_prob,
                    act=act,
                )
            elif model_type == 'gin':
                mp_layer = GraphIsomorphism(
                    hidden_dim=hidden_dim,
                    dropout_prob=dropout_prob,
                    act=act,
                    bias_mlp=bias_mlp
                )
            # GIN을 하는데 edge feature까지 함께쓰는 GIN_E 모델
            elif model_type == 'gin_e':
                mp_layer = GraphIsomorphismEdge(
                    hidden_dim=hidden_dim,
                    dropout_prob=dropout_prob,
                    act=act,
                    bias_mlp=bias_mlp
                )
            elif model_type == 'gat':
                mp_layer = GraphAttention(
                    hidden_dim=hidden_dim,
                    num_heads=num_heads,
                    dropout_prob=dropout_prob,
                    act=act,
                    bias_mlp=bias_mlp
                )
            else:
                # 예외처리: model_type을 잘 못 써줬을 경우
                raise ValueError('Invalid model type: you should choose model type in [gcn, gin, gin_e, gat, ggnn]')
                
			# self.mp_layer(nn.ModuleList)에 GCN(or GIN or GIN_E or GAT)을 num_layer=4번 넣음
            self.mp_layers.append(mp_layer)

        # linear transform으로 output이 하나의 스칼라 값이 나오도록 함
        self.linear_out = nn.Linear(hidden_dim, 1, bias=False)

        # is_classification: False
        self.is_classification = is_classification
        # classification이 True이면 sigmoid
        if self.is_classification:
            self.sigmoid = F.sigmoid 
    

	# forward() "graph를 받았을 때 어떻게 실행할 건지. 위에서 정의한 것들을 조합." 
    # 그래프가 들어왔을 때 prediction 값을 구하는 forward pass 하는 부분의 함수
    # training 하려면 true, training 안하려면 false (training=False라는 뜻은 dropout을 끄겠다는 의미)
    def forward(
            self, 
            graph,
            training=False,
        ):
		# graph의 initial node data (전체 node의 atom feature list)를 float형태로 변환 후 
        # embedding_node(linear layer로 linear transformation)한 후 h로 바꾸고
        h = self.embedding_node(graph.ndata['h'].float())
        e_ij = self.embedding_edge(graph.edata['e_ij'].float())
        # embedding한 node를 다시 graph의 node data로 넣어줌 
        graph.ndata['h'] = h
        graph.edata['e_ij'] = e_ij

		### Update the node features
        # message passing layer에다가 num_layers=4번 node feature들을 업데이트(graph를 업데이트) 해주는 파트. 
        for i in range(self.num_layers):            
            # 처음 graph가 ex)GCN(message passing layer)에 들어가서 업데이트 된 graph가 나오고, 그 graph가 또 GCN에 들어가서 graph가 업데이트 되는 방식. 
            # training=training : True
            graph = self.mp_layers[i](
                graph=graph,
                training=training
            )
                
		### Aggregate the node features and apply the last linear layer to compute the logit
        # 최종 업데이트된 graph에서 node feature들을 aggregation 해주고 'sum' aggregation으로 readout
        out = dgl.readout_nodes(graph, 'h', op=self.readout)
        # last linear layer를 통해서 graph를 하나의 스칼라값(:prediction 값)으로 업데이트
        out = self.linear_out(out)

        # classification이 True이면 sigmoid activaton function에 넣음 
        if self.is_classification:
            out = self.sigmoid(out)
        return out
